In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# Горячее кодирование слов или символов


Горячее кодирование - это самый распространенный, самый простой способ превратить токен в вектор. Вы уже видели это в действии в наших первоначальных примерах IMDB и Reuters из главы 3 (в нашем случае это были слова). Он состоит в том, чтобы связать уникальный целочисленный индекс с каждым словом, а затем превратить этот целочисленный индекс i в двоичный вектор размером N, размером словаря, который будет состоять из всех нулей, кроме i-й записи, которая будет 1 ,

Конечно, горячее кодирование может быть выполнено и на уровне символов. Чтобы однозначно понять, что такое горячая кодировка и как ее реализовать, вот вам два примера горячего кодирования: один для слов, другой для символов.



Горячее кодирование на уровне слов (пример):

In [3]:
import numpy as np

# This is our initial data; one entry per "sample"
# (in this toy example, a "sample" is just a sentence, but
# it could be an entire document).
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# First, build an index of all tokens in the data.
token_index = {}
for sample in samples:
    # We simply tokenize the samples via the `split` method.
    # in real life, we would also strip punctuation and special characters
    # from the samples.
    for word in sample.split():
        if word not in token_index:
            # Assign a unique index to each unique word
            token_index[word] = len(token_index) + 1
            # Note that we don't attribute index 0 to anything.

# Next, we vectorize our samples.
# We will only consider the first `max_length` words in each sample.
max_length = 10

# This is where we store our results:
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

Горячее кодирование на уровне символов (пример)

In [5]:
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable  # All printable ASCII characters.
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.

Обратите внимание, что в Keras есть встроенные утилиты для быстрого кодирования текста на уровне слов или символов, начиная с необработанных текстовых данных. Это то, что вы должны использовать, так как он позаботится о ряде важных функций, таких как удаление специальных символов из строк или включение первых N наиболее распространенных слов в вашем наборе данных (общее ограничение, чтобы избежать очень большие входные векторные пространства).

Использование Keras для кодирования на уровне слово:

In [7]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# We create a tokenizer, configured to only take
# into account the top-1000 most common words
tokenizer = Tokenizer(num_words=1000)
# This builds the word index
tokenizer.fit_on_texts(samples)

# This turns strings into lists of integer indices.
sequences = tokenizer.texts_to_sequences(samples)

# You could also directly get the one-hot binary representations.
# Note that other vectorization modes than one-hot encoding are supported!
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# This is how you can recover the word index that was computed
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.


Вариант одноразового кодирования - это так называемый трюк, который можно использовать, когда число уникальных токенов в вашем словаре слишком велико, чтобы обращаться с ним явно. Вместо того, чтобы явно присваивать индекс каждому слову и хранить ссылку на эти индексы в словаре, можно хешировать слова в векторы фиксированного размера. Обычно это делается функцией хеширования. Основным преимуществом этого метода является то, что он не поддерживает явный индекс слов, что экономит память и позволяет оперативно кодировать данные (начиная сразу же генерировать векторы токенов, прежде чем увидеть все доступные данные). Единственный недостаток этого метода заключается в том, что он подвержен «коллизиям хеша»: два разных слова могут заканчиваться одним и тем же хешем, и впоследствии любая модель машинного обучения, смотрящая на эти хэши, не сможет определить разницу между этими словами. Вероятность столкновений хэшей уменьшается, когда размерность пространства хеширования намного больше, чем общее количество хешируемых уникальных токенов.

Горячее кодирование на уровне слов с хешированием:

In [9]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# We will store our words as vectors of size 1000.
# Note that if you have close to 1000 words (or more)
# you will start seeing many hash collisions, which
# will decrease the accuracy of this encoding method.
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # Hash the word into a "random" integer index
        # that is between 0 and 1000
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.